In [1]:
%pylab inline
import scipy.optimize
import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

Populating the interactive namespace from numpy and matplotlib


In [2]:
import theano
from theano import tensor as T
from lasagne.updates import sgd, apply_momentum, nesterov_momentum, adagrad, adadelta

In [3]:
mnist = fetch_mldata('MNIST original', data_home='./data')
y_all = mnist.target[:, np.newaxis]
intercept = np.ones_like(y_all)
data = np.hstack([intercept, mnist.data, y_all])
np.random.shuffle(data)

In [4]:
def normalize_features(train, test):
    """Normalizes train set features to a standard normal distribution
    (zero mean and unit variance). The same procedure is then applied
    to the test set features.
    """
    train_mean = train.mean(axis=0)
    # +0.1 to avoid division by zero in this specific case
    train_std = train.std(axis=0) + 0.1
    
    train = (train - train_mean) / train_std
    test = (test - train_mean) / train_std
    return train, test

In [5]:
train_data_count = 60000
Xt = data[:train_data_count, :-1]
yt = data[:train_data_count, -1:].astype(int)


Xv = data[train_data_count:, :-1]
yv = data[train_data_count:, -1:].astype(int)

Xt, Xv = normalize_features(Xt, Xv)

In [6]:
binarizer = preprocessing.LabelBinarizer()
binarizer.fit(yt)
ytb = binarizer.transform(yt)

In [71]:
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape, m=0.01):
    w = np.random.randn(*shape) * m
    return theano.shared(floatX(w))

def model(X, W, B):
    h = X
    for w, b in zip(W[:-1], B[:-1]):
        h = T.nnet.sigmoid(T.dot(h, w) + b)
#     b = B[:-1]
#     w = W[:-1]
    h = T.dot(h, W[-1]) + B[-1]
    h = T.nnet.softmax(h)
    return h

In [72]:
Layers = [Xt.shape[1], 256, 10]

X = T.fmatrix()
Y = T.fmatrix()

W = []
B = []
for l1, l2 in zip(Layers[:-1], Layers[1:]):
    W.append(init_weights((l1, l2)))
    B.append(init_weights((l2,), m=1))



hwx = model(X, W, B)
cost = T.mean(T.nnet.categorical_crossentropy(hwx, Y))

params = W + B

updates = sgd(cost, params, learning_rate=0.1)


predict = theano.function(inputs=[X], outputs=hwx, allow_input_downcast=True)
train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)


In [58]:


X = T.fmatrix()
Y = T.fmatrix()

def model(X, w_h, b_h, w_o, b_o):
    h = X
    h = T.nnet.sigmoid(T.dot(h, w_h) + b_h)
    h = T.nnet.softmax(T.dot(h, w_o) + b_o)
    return h

w_h = init_weights((Xt.shape[1], 256))
b_h = init_weights((256,))
w_o = init_weights((256, 10))
b_o = init_weights((10,))

hwx = model(X, w_h, b_h, w_o, b_o)
cost = T.mean(T.nnet.categorical_crossentropy(hwx, Y))

params = [w_h, b_h, w_o, b_o]

updates = sgd(cost, params, learning_rate=0.1)


predict = theano.function(inputs=[X], outputs=hwx, allow_input_downcast=True)
train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)


In [73]:
for i in range(100):
    train(Xt, ytb)
    if i % 10 == 0:
        ytp = binarizer.inverse_transform(predict(Xt))
        print(i, 'Train acc', accuracy_score(yt, ytp))

0 Train acc 0.100766666667
10 Train acc 0.14955
20 Train acc 0.2194
30 Train acc 0.3319
40 Train acc 0.432416666667
50 Train acc 0.501416666667
60 Train acc 0.55075
70 Train acc 0.592716666667
80 Train acc 0.632016666667
90 Train acc 0.6697


In [20]:
ytp = binarizer.inverse_transform(predict(Xt))
print('Train acc', accuracy_score(yt, ytp))
yvp = binarizer.inverse_transform(predict(Xv))
print('Valid acc', accuracy_score(yv, yvp))

Train acc 0.765083333333
Valid acc 0.7655
